In [ ]:
"""

當機器人被加入群組，我們會收到 JoinEvent

此時我們可以取得 groupId

並可以利用 replyToken 來回應這一事件

ps. 記得要將 LINE 官方的機器人設定改為允許加入群組

"""

In [ ]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""

機器人被加入群組時觸發

向群組說聲早安

"""


from linebot.models import (
    JoinEvent, TextSendMessage,
)

@handler.add(JoinEvent)
def handle_join(event):
    print(event)
    
    print("群組 ID：" + event.source.group_id)
    
    reply_message_list = [
        TextSendMessage(text="大家早 ~\n我是來自火星的機器人，很榮幸加入各位的群組！"),
        TextSendMessage(text="題外話，你們知道每個群組都有一個 ID 嗎？"),
        TextSendMessage(text="我絕對不會告訴各位，你們的群組 ID 是 " + event.source.group_id)
    ]
    
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [ ]:

'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')